In [3]:
from fastapi import FastAPI, File, UploadFile, HTTPException, Request
from google.cloud import storage
from dotenv import load_dotenv
import os
load_dotenv()
from langchain.document_loaders import PDFMinerLoader
bucket_name = os.getenv("bucket_name")
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders.csv_loader import CSVLoader
import tempfile
import time
from langchain.llms import LlamaCpp
from langchain.callbacks.base import BaseCallbackHandler
import streamlit as st
from langchain.callbacks import StreamlitCallbackHandler
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
llm=LlamaCpp(
        model_path="./llama-2-7b-chat.Q5_K_S.gguf",
        n_gpu_layers=40,
        n_batch=512,
        n_ctx=4096,
        f16_kv=True,
        verbose=True,  # Verbose is required to pass to the callback manager
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3070, compute capability 8.6
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./llama-2-7b-chat.Q5_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 110

In [3]:
llm("Hello world")


llama_print_timings:        load time =     106.03 ms
llama_print_timings:      sample time =      33.27 ms /   256 runs   (    0.13 ms per token,  7693.46 tokens per second)
llama_print_timings: prompt eval time =     105.97 ms /     3 tokens (   35.32 ms per token,    28.31 tokens per second)
llama_print_timings:        eval time =    9922.16 ms /   255 runs   (   38.91 ms per token,    25.70 tokens per second)
llama_print_timings:       total time =   10476.05 ms


'! I am a machine learning enthusiast with a strong passion for AI and its applications in various industries. My name is Nithya, and I have completed my master’s degree in computer science with a specialization in machine learning.\nDuring my academic journey, I had the opportunity to work on several projects that involved developing and training machine learning models for different use cases. One of my most notable projects was a sentiment analysis project where we used natural language processing techniques to analyze customer reviews and predict their sentiment. This project helped me understand the power of machine learning in analyzing complex data sets and providing valuable insights.\nIn addition to my academic experience, I have also worked on several freelance projects that involved developing machine learning models for clients across different industries. These projects gave me hands-on experience in working with real-world datasets and solving practical problems using mac

In [4]:
df=CSVLoader('./TheDeal.csv')

In [5]:
db=FAISS.from_documents(df.load(),embedding=OpenAIEmbeddings())